In [394]:
# import the necessary packages
import numpy as np
import pandas as pd
import cv2
from matplotlib import pyplot as plt
import skimage
from scipy.signal import convolve2d as conv2
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
import re
import string

# Using google cloud vision

In [30]:
from google.cloud import vision
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "Salary Dost-a9e341ace734 - Copy.json"

In [73]:
#The Vision API can perform feature detection on a local image file by sending the contents
#of the image file as a base64 encoded string in the body of your request.
def detect_text(path):
    """Detects text in the file."""
    from google.cloud import vision
    import io
    client = vision.ImageAnnotatorClient()

    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.types.Image(content=content)

    response = client.text_detection(image=image)
    texts = response.text_annotations
    #print('Texts:')
    string=[]
    for text in texts:
        #print('\n"{}"'.format(text.description))
        string.append("{}".format(text.description))

        vertices = (['({},{})'.format(vertex.x, vertex.y) for vertex in text.bounding_poly.vertices])

        #print('bounds: {}'.format(','.join(vertices)))

    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))
    return string[0]

In [74]:
text=detect_text("sd2.jpg")

In [75]:
text1=detect_text("1.jpeg")
text1

'GOVERNMENT OF INDIA\nAyaz Pasha\nBES Oa / Year of Birth : 1993\naa / Male\n9579 7094 2866\n'

In [68]:
text

'இந்திய அரசாங்கம்\nGovernment of India\nwwrte et\nகருப்பையா\nKaruppaiya\nதந்தை : கருப்பையா\nFather : KARUPPAIYA\nபிறந்தவருடம்/Year of Birth: 1985\nஆண்பால் / Male\n2062 2196 5635\nஆதார் - சாதாரண மனிதனின் அதிகாரம்\nAt-par cheque Book\n'

In [45]:
#set GOOGLE_APPLICATION_CREDENTIALS=C:\Users\Lenovo\Desktop\Salary Dost-a9e341ace734.json

In [46]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "Salary Dost-a9e341ace734 - Copy.json"

In [53]:
#downloading images
data = pd.read_csv('Data_KYC.csv')
data.head()

,name,path,status,id
0,Aadhar Card Front,https://salarydostkycdocbucket.s3.ap-south-1.a...,approve,11421
1,Aadhar Card Front,https://salarydostkycdocbucket.s3.ap-south-1.a...,approve,36830
2,Aadhar Card Front,https://salarydostkycdocbucket.s3.ap-south-1.a...,approve,22734
3,Aadhar Card Front,https://salarydostkycdocbucket.s3.ap-south-1.a...,approve,30477
4,Aadhar Card Front,https://salarydostkycdocbucket.s3.ap-south-1.a...,approve,375


In [60]:
#after downloading images
import glob
#img_dir = "aadhar front" # Enter Directory of all images 
#data_path = os.path.join(img_dir,'*g')
#files = glob.glob(data_path)
#data = []
#for f1 in files:
#    img = cv2.imread(f1)
#    data.append(img)

In [550]:
# list the files in directory 
lis1 = os.listdir('C:\\Users'
                 '\\Lenovo\\ocr\\aadhar front') 
lis2 = os.listdir('C:\\Users'
                 '\\Lenovo\\ocr\\aadhar back')
lis3 = os.listdir('C:\\Users'
                 '\\Lenovo\\ocr\\pan')

#print(lis3) 

# Calling ocr function

In [551]:
ocr_af=[]
ocr_ab=[]
ocr_pan=[]

for i in range(0,len(lis1)):
    ocr_af.append(detect_text("aadhar front\\"+lis1[i]))
    

In [85]:
for i in range(0,len(lis2)):
    ocr_ab.append(detect_text("aadhar back\\"+lis2[i]))
    

In [87]:
for i in range(0,len(lis3)):
    ocr_pan.append(detect_text("pan\\"+lis3[i]))
    

# Formatting the text obtained

In [570]:
def f1(y):
    y = y.replace("-",":")
    y=re.sub("[^A-Za-z0-9:/\n#, ]", "", y.strip())
    y=y.strip().splitlines()
    
    y = [x for x in y if x != " " and x!="  " and x!=""]
    y = [x for x in y if x != '   ' and x!="    "]
    y = [x for x in y if x != ',']
    y = [x for x in y if x != ':']
    y = [x.strip() for x in y]
    y = [x.replace('SIO','S/O') for x in y]
    y = [x.replace('DIO','D/O') for x in y]
    y = [x.replace('DOR','DOB') for x in y]
    

    return y

In [631]:
def f2(z):
    for i in range(0,len(z)):
        index=0
        if "DOB" in z[i]:
            index = z[i].index('DOB')
            z[i]=z[i][index:]
        
        if "Female" in z[i]:
            index = z[i].index("Female")
            z[i]=z[i][index:index+6]
        
        if "Mele" in z[i]:
            index = z[i].index("Mele")
            z[i]=z[i][index:index+4]
            z[i]="Male"
            
        if "Male" in z[i]:
            index = z[i].index("Male")
            z[i]=z[i][index:index+4]
            
        if "MALE" in z[i]:
            index = z[i].index("MALE")
            z[i]=z[i][index:index+4]
            
        if "FEMALE" in z[i]:
            index = z[i].index("FEMALE")
            z[i]=z[i][index:index+6]
        
        if "Fomaie" in z[i]:
            index = z[i].index("Fomaie")
            z[i]=z[i][index:index+6]
            z[i]="Female"
            
            
        if "Enrollment No" in z[i]:
            index = z[i].index("Enrollment No")
            z[i]=z[i][index:]
            
        if "Enrolment No" in z[i]:
            index = z[i].index("Enrolment No")
            z[i]=z[i][index:]
            
        if "Year of Bith:" in z[i]:
            index = z[i].index("Year of Bith:")
            z[i]=z[i][index:]
            
        if "Year of Birth" in z[i]:
            index = z[i].index("Year of Birth")
            z[i]=z[i][index:]
            
        if "YoB" in z[i]:
            index = z[i].index("YoB")
            z[i]=z[i][index:]
        
        if "YB" in z[i]:
            index = z[i].index("YB")
            z[i]=z[i][index+2:]
            
        if "Your Aadhaar No" in z[i]:
            index = z[i].index("Your Aadhaar No")
            z[i]=z[i][index:]
            
            
        
    return z

In [438]:
#z1=re.sub("[^A-Za-z0-9:/\n_ -  ]", "", ocr_af[4].strip())

In [439]:
#z1.strip().splitlines()

In [633]:
af=[]
af1=[]
for i in range(0,len(ocr_af)):
    af.append(f1(ocr_af[i]))
    af1.append(f2(af[i]))


In [652]:
for i in range(0,len(af1)):
    af1[i]=[x for x in af1[i] if x!=":"]
    af1[i]=[x for x in af1[i] if x!=","]
    af1[i]=[x.replace("DOP","DOB") for x in af1[i]]
    af1[i]=[x.replace("DO8","DOB") for x in af1[i]]
    af1[i]=[x.replace("IOE","DOB") for x in af1[i]]
    af1[i]=[x.replace("DOB :","DOB:") for x in af1[i]]
    af1[i]=[x.replace("/DOB","DOB") for x in af1[i]]
    af1[i]=[x.replace("DOR","DOB") for x in af1[i]]
    af1[i]=[x.replace("YoB","YOB") for x in af1[i]]
    af1[i]=[x.replace("Year of Birth","YOB") for x in af1[i]]
    af1[i]=[x.replace("Year of Bith","YOB") for x in af1[i]]
    af1[i]=[x.replace("Mobila No","Mobile No") for x in af1[i]]
    
    #af1[i]=[x.replace("DOB","DOB") for x in af1[i]]
af1

[['Government of India',
  'KR Naveen Kumar',
  'DOB: 04/11/1990',
  'Male',
  '5380 6233 5723'],
 ['Government of India',
  'Jaishankar Rajamani Nadar',
  'YOB : 1975',
  'Male',
  '2814 5094 9451',
  '31'],
 ['GOVERNMENT OF INDIA',
  'Patel Hemantkumar Ramanlal',
  'YOB : 1979',
  'Male',
  '7642 9995 4742'],
 ['GOVERNMENTIORINDIA', 'Irfan', 'DOB: 01/01/1995', 'MALE', '2791 7151 1194'],
 ['To',
  'Arthi M',
  'D/O: Moorthy',
  '2/1250 A KARPAGAVINAYAGA NAGAR',
  '6TH CROSS STREET THORAIPAKKAM 193 WARD',
  'Karapakkam',
  'Oggiamthoraipakkam',
  'Sholinganallur Kancheepuram',
  'Tamil Nadu 600097',
  '9176172341',
  'MN511414333FT',
  'Your Aadhaar No',
  '6889 9338 6557',
  'Government of India',
  'Arthi M',
  'DOB: 21/11/1990',
  'Female',
  '6889 9338 6557',
  '141433',
  '03/10/2013'],
 ['Unique IdentificationAutherity of India',
  'Government of India',
  'Enrollment No : 2017/78225/22269',
  'Prashant Venkatesh Bandivaddar',
  'S/O Venkatesh Bandivaddar',
  '#37',
  'Haleshiden

# Creating lists for dataframe

In [825]:
gender=[]
dob=[]
name=[]
father=[]

for i in range(0,len(af1)):
    
    for j in range(0,len(af1[i])):
        
        if af1[i][j]=="Male" or af1[i][j]=="MALE":
            gender.append("Male")
            index=j
            
            
        if af1[i][j]=="Female" or af1[i][j]=="FEMALE":
            gender.append("Female")
            index=j
            
        
    
    dob.append(af1[i][index-1])
    
    if "Father" in af1[i][index-2]:
        father.append(af1[i][index-2])
    elif "Father" in af1[i][index-3]:
        father.append(af1[i][index-3])
    else:
        father.append("")
        
        
    if "DOB" in af1[i][index-2] or "YOB" in af1[i][index-2]:
        if "Father" not in af1[i][index-3]:
            name.append(af1[i][index-3])
            
        if "Father" in af1[i][index-3]:
            name.append(af1[i][index-4])
        #print(af1[i])
    else:
        if "Father" not in af1[i][index-2]:    
            name.append(af1[i][index-2])
            
        if "Father" in af1[i][index-2]:
            name.append(af1[i][index-3])
        
             
    
name            

['KR Naveen Kumar',
 'Jaishankar Rajamani Nadar',
 'Patel Hemantkumar Ramanlal',
 'Irfan',
 'Arthi M',
 'Prashant Venkatesh Bandivaddar',
 'Rehana Shamsher Shaikh',
 'lyengar',
 'Suneet Shukla',
 'Ravindra Satish Nayak',
 'Sanjeeb Rai',
 'Nikita Biswas',
 'Amol Sahebrao Nagare',
 'Ayaz Pasha',
 'Prashant Suresh Kadam',
 'Debashis Mondal',
 'Deepak Balasaheb Giram',
 'Rajendran Kumar',
 'Rahamathulla',
 'Sumit Ghosh',
 'Nitin Rajan Salvi',
 'Dattatray Sonu Rane',
 'Ravindra Ramrao Makone',
 'BISWA RANJAN JENA',
 'Azad Singh',
 'Bhola Han Punchal',
 'Niladri Bhusan Panigrahi',
 'Sundara Moorthi',
 'Pankaj Chouhan',
 'Rajnikant Tukaram Kamble',
 'Shivraj Dohar',
 'Shrikrishna',
 'Mallikarjuna KP',
 'S Kishore Kumar',
 'Mobin Salim Patel',
 'Mohammad Sameer',
 'Sambhu Thakur',
 'Sunil Kumar U',
 'Firoz Kasim Shaikh',
 'Rajesh Bhaskar Kanitkar',
 'V Balasubramaniam',
 'Khatavkar',
 'Mirza Mujahid Ali Baig',
 'Buddhadeb Santra',
 'Pravesh Lathiyan',
 'Raghavendra Prasad',
 'Shaik Yasin',
 'M

In [789]:
len(name)

191

In [654]:
dob1=[]
for i in range(0,len(dob)):
    if "DOB" in dob[i]:
        index = dob[i].index('DOB')
        dob1.append(dob[i][index+4:].strip())
    
    elif "YOB" in dob[i]:
        index = dob[i].index('YOB')
        dob1.append(dob[i][index+4:].strip())
    
    else:
        dob1.append(dob[i])
        
#dob1=[x[1:].strip() for x in dob1 if x[0]==":"]
for i in range(0,len(dob1)):
    if dob1[i][0]==":":
        dob1[i]=dob1[i][1:].strip()
    if dob1[i][0]=="/":
        dob1[i]=dob1[i][7:].strip()
        
dob1

['04/11/1990',
 '1975',
 '1979',
 '01/01/1995',
 '21/11/1990',
 '06/01/1995',
 '13/07/1992',
 '24/09/1976',
 '31/12/1977',
 '15/04/1974',
 '04/09/1992',
 '26/09/1991',
 '11/11/1986',
 '1993',
 '17/05/1988',
 '12/12/1987',
 '03/04/1994',
 '1986',
 '1976',
 '17/12/1996',
 '06/01/1983',
 '29/01/1984',
 '11:09:1988',
 '26/06/1979',
 '01/01/1996',
 'UN4/1994',
 '06/03/1987',
 '05/05/1989',
 '06/05/1990',
 '01/01/1993',
 '15/04/1988',
 '1984',
 '25/04/1981',
 '1985',
 '14/05/1985',
 '20/05/1994',
 '06/07/1968',
 '1979',
 '20/01/1989',
 '1973',
 '08/08/1976',
 '30/09/1987',
 '1983',
 '16/08/1989',
 '16/07/1994',
 '07no/1985',
 '05/02/1981',
 '10/05/1973',
 '27/09/1992',
 '15/03/1991',
 '28/11/1977',
 '1986',
 '1982',
 '01/10/1996',
 '02/03/1984',
 '25/03/1983',
 '14/06/1979',
 '01/01/1988',
 '1983',
 '24/06/1986',
 '1966',
 '27/12/1988',
 '06/01/1988',
 '14/12/1993',
 '30/05/1987',
 '1995',
 '03/09/1981',
 '01/01/1972',
 '31/01/1993',
 '01/06/1986',
 '17/07/1991',
 '20/10/1994',
 '30/01/1989'

In [790]:
len(dob1)

191

aadhar=[]
for i in range(0,len(af1)):
    for j in range(0,len(af1[i])):
        if "MALE" in af1[i][j]:
            if "DOB" in af1[i][j-1]:
                if "Mobile No" in af1[i][j+1]:
                    aadhar.append(af1[i][j+2])
                else:
                    aadhar.append(af1[i][j+1])
                    
            elif "YOB" in af1[i][j-1]:
                    if "Mobile No" in af1[i][j+1]:
                        aadhar.append(af1[i][j+2])
                    else:
                        aadhar.append(af1[i][j+1])
                        
            else:
                if len(af1[i][j-1])==4 or len(af1[i][j-1])==10:
                    if "Mobile No" in af1[i][j+1]:
                        aadhar.append(af1[i][j+2])
                    else:
                        aadhar.append(af1[i][j+1])
                        
                
        else:
            if "Male" in af1[i][j]:
                if "DOB" in af1[i][j-1]:
                    if "Mobile No" in af1[i][j+1]:
                        aadhar.append(af1[i][j+2])
                    else:
                        aadhar.append(af1[i][j+1])
                        
                elif "YOB" in af1[i][j-1]:
                    if "Mobile No" in af1[i][j+1]:
                        aadhar.append(af1[i][j+2])
                    else:
                        aadhar.append(af1[i][j+1])
                        
                else:
                    if len(af1[i][j-1])==4 or len(af1[i][j-1])==10:
                        if "Mobile No" in af1[i][j+1]:
                            aadhar.append(af1[i][j+2])
                        else:
                            aadhar.append(af1[i][j+1])
                        
                        
        if "Female" in af1[i][j]:
            if "DOB" in af1[i][j-1]:
                    if "Mobile No" in af1[i][j+1]:
                        aadhar.append(af1[i][j+2])
                    else:
                        aadhar.append(af1[i][j+1])
                        
            elif "YOB" in af1[i][j-1]:
                    if "Mobile No" in af1[i][j+1]:
                        aadhar.append(af1[i][j+2])
                    else:
                        aadhar.append(af1[i][j+1])
                        
            else:
                if len(af1[i][j-1])==4 or len(af1[i][j-1])==10:
                    if "Mobile No" in af1[i][j+1]:
                        aadhar.append(af1[i][j+2])
                    else:
                        aadhar.append(af1[i][j+1])
                        
        if af1[i][j]=="klyn":
            aadhar.append(af1[i][j+1][2:])
        

# List for aadhar

In [802]:
aadhar=[]
mobile_no=[]
for i in range(0,len(af1)):
    if "Your Aadhaar No :" in af1[i]:
        index = af1[i].index("Your Aadhaar No :")
        aadhar.append(af1[i][index+1])
        mobile_no.append("")
    
    
    elif "MALE" in af1[i]:
        index = af1[i].index("MALE")
            #aadhar.append(af1[i][index+1])
        if "Mobile No" in af1[i][index+1]:
            aadhar.append(af1[i][index+2])
            mobile_no.append(af1[i][index+1])
        else:
            aadhar.append(af1[i][index+1])
            mobile_no.append("")
         
    elif "Male" in af1[i]:
        index = af1[i].index("Male")
        if "Mobile No" in af1[i][index+1]:
            aadhar.append(af1[i][index+2])
            mobile_no.append(af1[i][index+1])
        else:
            aadhar.append(af1[i][index+1])
            mobile_no.append("")
                        
                    
    elif "Female" in af1[i]:
        index = af1[i].index("Female")
            
        if "DOB" in af1[i][index-1]:
            if "Mobile No" in af1[i][index+1]:
                aadhar.append(af1[i][index+2])
                mobile_no.append(af1[i][index+1])
            else:
                aadhar.append(af1[i][index+1])
                mobile_no.append("")
                    
        elif "YOB" in af1[i][index-1]:
            if "Mobile No" in af1[i][index+1]:
                aadhar.append(af1[i][index+2])
                mobile_no.append(af1[i][index+1])
            else:
                aadhar.append(af1[i][index+1])
                mobile_no.append("")
                    
        else:
            if len(af1[i][index-1])==4 or len(af1[i][index-1])==10:
                if "Mobile No" in af1[i][index+1]:
                    aadhar.append(af1[i][index+2])
                    mobile_no.append(af1[i][index+1])
                else:
                    aadhar.append(af1[i][index+1])
                    mobile_no.append("")
                    
                
                        
    else:
        aadhar.append("")
        
        mobile_no.append("")
                    
       

In [804]:
len(mobile_no)
len(aadhar)

191

In [805]:
aadhar

['5380 6233 5723',
 '2814 5094 9451',
 '7642 9995 4742',
 '2791 7151 1194',
 '6889 9338 6557',
 '2909 0952 6799',
 '7652 0810 9325',
 '5959 0908 0412',
 '4345 6262 5439',
 '8924 3272 1962',
 '9712 1369 7275',
 '8397 0943 8292',
 '3519 0660 8310',
 '9579 7094 2866',
 '8339 2201 1893',
 '9423 8434 4259',
 '9435 9641 3360',
 '8676 8312 3768',
 '3486 7653 2052',
 '4141 2202 5868',
 '2423 4522 2232',
 '3062 9132 3185',
 '3388 8937 7006',
 '3122 1372 6401',
 '5973 8321 1000',
 '6114 7590 4489',
 '6831 4414 4375',
 '3330 3739 6411',
 '7608 9150 2158',
 '5669 8319 2306',
 '7370 8227 1447',
 '720 88125803',
 '2050 5714 5051',
 '5385 8864 1764',
 '4903 2480 8414',
 '8073 9162 2065',
 '6697 6994 5353',
 '5893 4965 4424',
 '7382 9030 6705',
 '9765 3378 6978',
 '7521 7388 7766',
 '5524 5207 0921',
 '2616 4797 5042',
 '2552 14345598',
 '5314 0996 8287',
 '8337 2248 9067',
 '6366 1553 8987',
 '2987 8275 5782',
 '2745 8584 0213',
 '5229 0148 8703',
 '5197 5327 6940',
 '6602 7785 7702',
 '7932 4624 509

In [724]:
len(af1)

191

In [806]:
mobile_no

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'Mobile No: 8970346001',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'Mobile No: 7738195887',
 '',
 '',
 'Mobile No: 7204233267',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'Mobile No 9830427425',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'Mobile No: 9533886695',
 'Mobile No: 8860621252',
 '',
 '',
 '',
 'Mobile No: 8600881905',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',

In [826]:
father=[x[7:].strip() for x in father]
father

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 ': Kumar',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 ': Rajendra Singh Manral',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 ': Karunakara Shetty',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 ': Nagaraj',
 '',
 '',
 '',
 '',
 '',
 '',
 ': KARUPPAIYA',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'Sanjay Narayan',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'HEMANT SAHU',
 '',
 '',
 '',
 '',
 '',
 '',
 '',


In [827]:
aadhar_front = pd.DataFrame(
    {'Name': name,
     'Gender': gender,
     'DOB/YOB':dob1,
     "Father's name":father,
     'Aadhar No.':aadhar,
     'Mobile No.':mobile_no,
     
     
    })


In [808]:
len(name)

191

In [809]:
len(gender)

191

In [811]:
len(dob1)

191

In [812]:
len(father)

191

In [813]:
father

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'Father : Kumar',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'Father : Rajendra Singh Manral',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'Father : Karunakara Shetty',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'Father : Nagaraj',
 '',
 '',
 '',
 '',
 '',
 '',
 'Father : KARUPPAIYA',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'Father: Sanjay Narayan',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'Father

In [830]:
aadhar_front.to_csv("aadhar_front.csv")

In [831]:
aadhar_front

,Name,Gender,DOB/YOB,Father's name,Aadhar No.,Mobile No.
0,KR Naveen Kumar,Male,04/11/1990,,5380 6233 5723,
1,Jaishankar Rajamani Nadar,Male,1975,,2814 5094 9451,
2,Patel Hemantkumar Ramanlal,Male,1979,,7642 9995 4742,
3,Irfan,Male,01/01/1995,,2791 7151 1194,
4,Arthi M,Female,21/11/1990,,6889 9338 6557,
...,...,...,...,...,...,...
186,Gautam,Male,01/08/1988,,7155 1322 8554,Mobile No: 8375018672
187,E Pushpan Kumar,Male,10/04/1981,,XXXX XXXX 5638,
188,Donekela Harish Goud,Male,260w19S,,7032 6666 2144,
189,Mukesh Kumar Suthar,Male,26/04/1992,,5099 0794 4019,
